<a href="https://colab.research.google.com/github/ele9996/Heart-failure-clinical-records-analysis/blob/master/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


#Import useful libraries


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

#Dataset visualization and data exploration
